In [7]:
import os
import sys

In [8]:
sys.path.append('../lib')

In [9]:
import cov_independence_test
import hsic_independence_test
import gen_processes

In [10]:
from importlib import reload

In [11]:
reload(cov_independence_test);
reload(hsic_independence_test);
reload(gen_processes);

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.ar_model as ar
import torch
torch.set_default_dtype(torch.double)
import sigkernel

In [13]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [20]:
import matplotlib.pyplot as plt
import pickle

In [21]:
rng = np.random.default_rng(1234)

In [22]:
with open('pickles/comparison_ds_for_normalized_sig_pde_kernel.pickle', 'rb') as inf:
    INPUT = pickle.load(inf)

In [39]:
paths_ll = torch.tensor(INPUT['processes_lead_lag'])

In [19]:
def psi(x, M=4, a=1):
    x = x ** 2

    if x <= M:
        return x

    return M + M ** (1 + a) * (M ** (-a) - x ** (-a)) / a

In [26]:
print(INPUT['comment'])
print(paths_ll.shape)

Process indices: Sample, Time, Lead/Lag
(20, 30, 2)


In [32]:
sig_kernel = sigkernel.SigKernel(sigkernel.LinearKernel(), 1)

In [38]:
for i in range(2, 30):
    print(paths_ll[0:1, :i+1, :].shape[2])

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [40]:
kernel_vs_time = [sig_kernel.compute_kernel(paths_ll[0:1, :i+1, :], paths_ll[0:1, :i+1, :]) for i in range(1, 30)]

In [59]:
normalized_kernel_vs_time = [psi(torch.sqrt(k)) for k in kernel_vs_time]

In [51]:
def get_tv_of_path(path):
    lin_ker_values = paths_ll[0] @ paths_ll[0].T
    vct_of_sq_distances = torch.diagonal(lin_ker_values)[:-1] + torch.diagonal(lin_ker_values)[1:] - 2*torch.diagonal(lin_ker_values, offset = 1)
    return torch.sqrt(vct_of_sq_distances).sum()

In [65]:
tv_path = get_tv_of_path(paths_ll[0, :, :]).item()

In [75]:
tv_path

9.9237479739895

In [96]:
kernel_vs_time[-1]

tensor([15.6600])

In [64]:
kernel_last_ts = normalized_kernel_vs_time[-1].item()

In [95]:
kernel_last_ts

6.9782870722007795

In [70]:
import scipy as scp
from scipy.optimize import brentq
from scipy.special import iv

In [92]:
f = lambda lam: 12.3 - iv(0, 2*12.3*lam)

In [93]:
brentq(f,  0, 1.001)

0.8649975046483256

In [101]:
12.3 - iv(0, 0.96287837*tv_path*2)

-18317778.53106583